In [1]:
from utility_bfvariater import *
import utility_common as common

# creat a BF variater
er = BFVariater()

# 1. Central Value

In [2]:
def saveCentralValues(er):
    print( "save conteral values")

    contralValue = []
    for i in range(4):

        ndata = er.ndata[i]
        nmcbg = er.nmcbg[i].sum(axis=1)
        nfake = er.nfake[i]
        
        a = np.sum(er.a[i] * er.ca, axis=1)

        slv0 = BFSolver3D(a)
        br0 = slv0.solveQuadEqn(slv0.setMeasuredX(nData=ndata, nMcbg=nmcbg+nfake))
        contralValue.append(br0)
    contralValue = np.array(contralValue).reshape(12)

    baseDir = common.getBaseDirectory() 

    print(contralValue)

    np.save(baseDir+'/data/combine/centralValue',contralValue)
saveCentralValues(er)

save conteral values
[0.10828633 0.11000734 0.10026387 0.11222122 0.11112967 0.10034579
 0.11010984 0.10841491 0.10217137 0.10807917 0.11012079 0.10717654]


# 2. uncertainty

### 2.1 Statistical Uncertainty

In [3]:
def saveStatError(er):
    print( "relative statistical uncertainty [%]")

    stat_sigma, stat_covar = [],[]

    for source in ["data","mcbg","mcsg"]:
        sigma, variance_raw = er.errStat(source,returnCovar=True)
        # reshape 4x3x3 to 12x12
        variance = np.zeros([12,12])
        for c in range(4):
            for i in range(3):
                for j in range(3):
                    variance[c*3+i,c*3+j] = variance_raw[c,i,j]
                    
        stat_sigma.append(sigma)
        stat_covar.append(variance)

    stat_sigma = np.array(stat_sigma)  
    stat_covar = np.array(stat_covar) 

    er.io_printErrorForExcelFormat(stat_sigma.transpose(1,0,2))

    s = stat_sigma.shape
    stat_sigma = stat_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/stat_sigma',stat_sigma)
    np.save(baseDir+'/data/combine/stat_covar',stat_covar)

saveStatError(er)

relative statistical uncertainty [%]
0.540,0.530,1.237, 0.710,0.634,1.484, 0.739,0.554,1.512, 0.899,0.703,1.797
0.177,0.741,0.763, 0.110,0.408,0.498, 0.892,0.255,1.060, 0.491,0.137,0.518
0.167,0.150,0.413, 0.188,0.164,0.426, 0.216,0.175,0.500, 0.231,0.191,0.517


### 2.2 Systematical Uncertainty

In [4]:
def saveSystError(er):
    print( "relative systematics uncertainty [%]")
    syst_sigma, syst_covar = [],[]

    for source in ["BtmUp","BteUp"]:
        sigma = er.errConstent(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    for source in ["lumin","mctt","mctw","mctother","mcw",'mcz','mcg','mcvv',
                   "fakee","fakemu","fakeetau","fakemutau"]:
        sigma = er.errSystem_crossSection(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
        
    for source in ["TopPtReweight","ERecoEff","EIDEff","MuRecoEff","MuIDEff","TauIDEff","JetToTauIDEff",
                   "EPt","MuPt","Tau0Pt","Tau1Pt","Tau10Pt","ETriggerEffProbeSyst","ETriggerEffTagSyst"
                  ]:
        sigma = er.errSystem_downVariation(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    for source in ["tauReweight1000","tauReweight11000","tauReweight21000",
                   "tauReweight3000","tauReweight13000"
                  ]:
        sigma = er.errSystem_upDownVariation(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
        
    for source in ["Pileup","JES","JER","BTag","Mistag"]:
        sigma = er.errSystem_upDownVariation(source)
        variance = np.outer(sigma.reshape(1,12), sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)   

    for source in ["fsr","isr","ue","meps"]:#
        sigma = er.errSystem_upDownVariation(source)
        variance = np.outer(sigma.reshape(1,12), sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)  
    
#     for source in ["fsr","isr","ue","meps"]:
#         sigma = er.errSystem_upDownVariationCutResidue(source,cutResidue=3)
#         variance = np.outer(sigma.reshape(1,12), sigma.reshape(1,12))
#         syst_sigma.append(sigma)
#         syst_covar.append(variance) 
    
    syst_sigma = np.array(syst_sigma) 
    syst_covar = np.array(syst_covar) 

    er.io_printErrorForExcelFormat(syst_sigma.transpose(1,0,2))

    s = syst_sigma.shape
    syst_sigma = syst_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/syst_sigma',syst_sigma)
    np.save(baseDir+'/data/combine/syst_covar',syst_covar)

saveSystError(er)

relative systematics uncertainty [%]
-0.002,-0.019,0.029, -0.002,-0.019,0.029, -0.003,-0.019,0.029, -0.003,-0.019,0.030
-0.047,-0.017,0.098, -0.047,-0.017,0.098, -0.041,-0.013,0.100, -0.043,-0.013,0.105
0.328,-0.459,0.120, 0.092,-0.064,0.048, -0.134,0.388,0.203, 0.002,0.100,0.092
0.002,0.000,0.150, -0.009,-0.014,0.032, 0.021,0.011,0.147, -0.011,-0.002,-0.003
-0.002,-0.001,-0.157, 0.009,0.015,-0.033, -0.022,-0.012,-0.154, 0.011,0.002,0.004
0.062,0.061,0.032, 0.053,0.052,0.058, 0.062,0.060,0.032, 0.052,0.053,0.039
0.341,0.352,0.323, 0.068,0.067,0.065, 0.347,0.345,0.364, 0.064,0.066,0.084
0.492,-2.641,-0.236, 0.121,-0.489,-0.055, -1.567,0.498,-0.172, -0.273,0.103,0.041
0.019,0.019,0.028, 0.005,0.005,0.007, 0.248,0.246,0.212, 0.058,0.058,0.080
-0.004,-0.044,-0.026, 0.001,-0.010,-0.005, -0.038,-0.003,-0.021, -0.008,0.001,-0.001
0.000,0.000,0.000, 0.000,0.000,0.000, 1.157,1.112,2.397, 0.218,0.217,0.404
0.738,0.733,1.554, 0.222,0.213,0.382, 0.000,0.000,0.000, 0.000,0.000,0.000
0.000,0.000,0.0

# 3. output Latex Table

In [5]:
%run -i '../scripts/note_uncertainty.py'

In [10]:
#     for source in ["e","mu","tauID",'jetMisTauID']:
#         sigma = er.errSystem_objectEff(source)
#         variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
#         syst_sigma.append(sigma)
#         syst_covar.append(variance)

In [11]:
sigma = er.errSystem_upDownVariationCutResidue("fsr",3)
sigma/0.108*100

array([[-0.72937743, -0.51049613, -1.02998876],
       [-0.27851618, -0.24015098, -0.80058707],
       [-0.08171089, -0.39995758, -2.18621547],
       [-0.96873063,  0.00758264, -1.85801155]])

In [12]:
sigma = er.errSystem_upDownVariation("ue")
sigma/0.108*100

array([[-0.02107514, -0.03684919,  1.66459745],
       [-0.30556739,  1.01660794,  0.26593908],
       [ 0.12169331,  0.17722437,  1.06014275],
       [ 0.17243485, -0.1332533 ,  0.05264139]])

In [13]:
sigma = er.errSystem_upDownVariationCutResidue("meps",3)
sigma/0.108*100

array([[0.77518768, 0.77455279, 1.62759062],
       [0.75296454, 0.73357212, 1.30123416],
       [0.46505114, 0.82507354, 1.70522452],
       [0.72501211, 0.71365991, 1.29238623]])

In [14]:
counts = pd.read_pickle( "../data/counts/count_.pkl")

In [77]:
counts0 = pd.read_pickle( "../data/counts/count_isrDown.pkl")
counts1 = pd.read_pickle( "../data/counts/count_isrUp.pkl")

In [68]:
tr,ch,mc = 0,3,1

acc = common.matrixToArray(counts.acc[tr][ch][mc])
accUp = common.matrixToArray(counts1.acc[tr][ch][mc])
accDown = common.matrixToArray(counts0.acc[tr][ch][mc])

accVar = common.matrixToArray(counts.accVar[tr][ch][mc])
accUpVar = common.matrixToArray(counts1.accVar[tr][ch][mc])
accDownVar = common.matrixToArray(counts0.accVar[tr][ch][mc])

resUp = abs(accUp-acc)/(accVar+accUpVar)**0.5
resDown = abs(accDown-acc)/(accVar+accDownVar)**0.5

name = common.WWBranchNames()
for i in range(21):
    sfUp,sfDown = 0,0
    if resUp[i] > 3:
        sfUp = accUp[i]/acc[i]
        sfDown = accDown[i]/acc[i]
    print("{:20}: {:5.3f}, {:5.3f}".format(name[i],sfUp,sfDown))

$ee$                : 0.000, 0.000
$\mu\mu$            : 0.962, 1.010
$e\mu$              : 0.945, 1.022
$\tau_e\tau_e$      : 0.000, 0.000
$\tau_\mu\tau_\mu$  : 0.000, 0.000
$\tau_e\tau_\mu$    : 0.000, 0.000
$\tau_e\tau_h$      : 0.000, 0.000
$\tau_\mu\tau_h$    : 0.000, 0.000
$\tau_h\tau_h$      : 0.000, 0.000
$e\tau_e$           : 0.000, 0.000
$e\tau_\mu$         : 0.000, 0.000
$e\tau_h$           : 0.000, 0.000
$\mu\tau_e$         : 0.920, 1.040
$\mu\tau_\mu$       : 0.943, 1.023
$\mu\tau_h$         : 0.931, 1.033
$eh$                : 0.528, 1.388
$\mu h$             : 0.916, 1.051
$\tau_eh$           : 0.000, 0.000
$\tau_\mu h$        : 0.920, 1.060
$\tau_hh$           : 0.573, 1.419
$hh$                : 0.502, 1.457


In [88]:
tr = 3
acc = counts.acc[tr].copy()
acc1 = counts1.acc[tr].copy()
acc0 = counts0.acc[tr].copy()

acc01 = (acc1*1/3+acc0*2/3)

accVar = counts.accVar[tr].copy()
acc1Var = counts1.accVar[tr].copy()
acc0Var = counts0.accVar[tr].copy()

res1 = abs(acc1-acc)/(accVar+acc1Var)**0.5
res0 = abs(acc0-acc)/(accVar+acc0Var)**0.5
res01 = abs(acc0-acc1)/(acc1Var+acc0Var)**0.5

for ch in range(4):
    mc=1
    for i in range(6):
        for j in range(6):
            if res01[ch,mc][i,j]<3:
                acc1[ch,mc][i,j] = acc01[ch,mc][i,j]
                acc0[ch,mc][i,j] = acc01[ch,mc][i,j]

In [89]:
# up tuning
slv1 = BFSolver3D( np.sum(acc1*er.ca,axis=1) )
BW1  = slv1.solveQuadEqn(slv1.setMeasuredX(nData=counts1.ndata[tr], nMcbg=counts1.nmcbg[tr].sum(axis=1)+counts1.nfake[tr]))
# down tuning
slv2 = BFSolver3D( np.sum(acc0*er.ca,axis=1) )
BW2  = slv2.solveQuadEqn(slv2.setMeasuredX(nData=counts0.ndata[tr], nMcbg=counts0.nmcbg[tr].sum(axis=1)+counts0.nfake[tr]))
# differentce between up and down tuning
print((BW1-BW2)/2/0.1080*100)

[0.0988434  0.09565056 0.17098258]


In [85]:
print(counts.ndata[0]-counts0.ndata[0])
print(counts.nmcbg[0]-counts0.nmcbg[0])
print(counts.nfake[0]-counts0.nfake[0])

[0. 0. 0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0.]


In [24]:
counts1.accVar[0][3][1]**0.5/counts1.acc[0][3][1]

array([[1.23812276e+00, 5.44267664e-03,            inf, 2.26110848e-02,
        6.02663712e-01, 1.20661118e-01],
       [5.44267664e-03, 7.80294975e-03, 1.27426556e-02, 1.38241656e-02,
        5.41245226e-03, 1.07055726e-03],
       [           inf, 1.27426556e-02,            inf, 5.33164419e-02,
                   inf, 2.40599071e-01],
       [2.26110848e-02, 1.38241656e-02, 5.33164419e-02, 6.28052574e-02,
        2.26312636e-02, 4.59895401e-03],
       [6.02663712e-01, 5.41245226e-03,            inf, 2.26312636e-02,
        4.35954570e-01, 1.03693105e-01],
       [1.20661118e-01, 1.07055726e-03, 2.40599071e-01, 4.59895401e-03,
        1.03693105e-01, 3.80247767e-02]])

In [2]:
sigma, variance_raw = er.errStat("mcsg",returnCovar=True)

In [4]:
sigma/0.108*100

array([[0.53401089, 0.45867545, 1.15690868],
       [0.95222688, 0.80551395, 1.82853771],
       [0.64549883, 0.52774019, 1.38175311],
       [1.2019165 , 0.93939064, 2.2449766 ]])